In [14]:
# Settings

onColab = True

# Import a preprocess_input function or define one yourself
# from tensorflow.keras.applications.resnet50 import preprocess_input

# e.g. definition of preprocFun:
def preprocess_input(x):
  return x / 255

model_dir = 'baseline_best'

# if false use evaluate fun, if true use for loop with predict
evaluate_with_for_loop = False

validation_split = 0.2

# only care if not evaluating with for loop:
batch_size = 64

In [15]:
if onColab:
  from google.colab import drive
  drive.mount('/gdrive')
  %cd /gdrive/MyDrive/University/ANN/CHALLENGE1/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/University/ANN/CHALLENGE1


In [16]:
import os
import time
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator

tfk = tf.keras
tfkl = tf.keras.layers
seed = 20

In [17]:
#@title model.py:
import os
import tensorflow as tf
#@markdown **REMEMBER TO DEFINE preprocess_input**

#@markdown **REMEMBER TO DEFINE model_dir**

class model:
    def __init__(self, path):
        self.model = tf.keras.models.load_model(os.path.join(path, model_dir))

    def predict(self, X):
        out = self.model.predict(preprocess_input(X))
        out = tf.argmax(out, axis=-1)

        return out

In [ ]:
a = model("./")

In [18]:
isTest = True
dataset_path = './training'
img_h, img_w = (256, 256)
test_path = './test'

labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

def load_data(dontUseFun=True, fun=None):
  image_generator = ImageDataGenerator(preprocessing_function = fun, validation_split=validation_split)  
  if dontUseFun:
    image_generator = ImageDataGenerator(validation_split=validation_split)  


  # Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
  train_gen = image_generator.flow_from_directory(directory=dataset_path,
                                                target_size=(img_h, img_w),
                                                color_mode='rgb',
                                                classes=labels, # can be set to None
                                                class_mode='categorical',
                                                subset='training',
                                                batch_size=batch_size,
                                                shuffle=True,
                                                seed=seed)

  valid_gen = image_generator.flow_from_directory(directory=dataset_path,
                                                target_size=(img_h, img_w),
                                                color_mode='rgb',
                                                classes=labels, # can be set to None
                                                class_mode='categorical',
                                                subset='validation',
                                                batch_size=batch_size,
                                                shuffle=False,
                                                seed=seed)
  test_gen = None
  if isTest:
    test_image_gen = ImageDataGenerator(preprocessing_function = fun)
    if dontUseFun:
      test_image_gen = ImageDataGenerator()
    test_gen = test_image_gen.flow_from_directory(directory=test_path,
                                                target_size=(img_h, img_w),
                                                color_mode='rgb',
                                                classes=labels, # can be set to None
                                                class_mode='categorical',
                                                batch_size=batch_size,
                                                shuffle=False,
                                                seed=seed)
  return train_gen, valid_gen, test_gen


# reload data without preprocessing
train_gen, valid_gen, test_gen = load_data(False, preprocess_input)

if not evaluate_with_for_loop:
  a.model.evaluate(test_gen, return_dict=True)

Found 11815 images belonging to 14 classes.
Found 2944 images belonging to 14 classes.
Found 2800 images belonging to 14 classes.
 6/44 [===>..........................] - ETA: 11:13 - loss: 0.0358 - accuracy: 0.9948

KeyboardInterrupt: ignored

In [ ]:
if evaluate_with_for_loop:
  batch_size=1
  train_gen, valid_gen, test_gen = load_data(True)
  n = next(test_gen)
  num = 0
  d =0
  while n:
    n = next(test_gen)
    pred, target = n[0], n[1]

    if a.predict(pred) == tf.argmax(target, axis=-1): num+=1
    d+=1
    if d % 64 == 0: print(d, ' iteration:', num/d)